In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import argparse
import torchvision.transforms as transforms
import cv2
from skimage import io

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/optical_flow_RAFT/RAFT/core')
from argparse import ArgumentParser
import argparse
from collections import OrderedDict

import numpy as np
from tqdm import tqdm
from raft import RAFT
from utils import flow_viz
fourcc = cv2.VideoWriter_fourcc(*'XVID')

def frame_preprocess(frame, device):
    frame = torch.from_numpy(frame).permute(2, 0, 1).float()
    frame = frame.unsqueeze(0)
    frame = frame.to(device)
    return frame

def vizualize_flow(img, flo, save, counter, vid_number):
    # permute the channels and change device is necessary
    img = img[0].permute(1, 2, 0).cpu().numpy()
    flo = flo[0].permute(1, 2, 0).cpu().numpy()
    # map flow to rgb image
    flo = flow_viz.flow_to_image(flo)
    flo = cv2.cvtColor(flo, cv2.COLOR_RGB2BGR)
    # flo = cv2.cvtColor(flo, cv2.COLOR_RGB2GRAY)
    # out.write(flo)
    # print("frame", counter)
    # concatenate, save and show images
    # img_flo = np.concatenate([img, flo], axis=0)
    if save:
        cv2.imwrite(f"/content/drive/MyDrive/AnomalyResearch/TrainOpticalFlow/Train{str(vid_number).zfill(3)}/frame_{str(counter+1).zfill(3)}.jpg", flo)
    # cv2_imshow(img_flo / 255.0)
    k = cv2.waitKey(25) & 0xFF
    if k == 27:
        return False
    return True


def get_cpu_model(model):
    new_model = OrderedDict()
    # get all layer's names from model
    for name in model:
        # create new name and update new model
        new_name = name[7:]
        new_model[new_name] = model[name]
    return new_model


def inference(args):
    # get the RAFT model
    model = RAFT(args)
    # load pretrained weights
    pretrained_weights = torch.load(args.model)

    save = args.save
    vid_number = args.video_number
    if save:
        if not os.path.exists(f"/content/drive/MyDrive/AnomalyResearch/TrainOpticalFlow/Train{str(vid_number).zfill(3)}"):
            os.mkdir(f"/content/drive/MyDrive/AnomalyResearch/TrainOpticalFlow/Train{str(vid_number).zfill(3)}")

    if torch.cuda.is_available():
        device = "cuda"
        # parallel between available GPUs
        model = torch.nn.DataParallel(model)
        # load the pretrained weights into model
        model.load_state_dict(pretrained_weights)
        model.to(device)
        print("in device")
    else:
        device = "cpu"
        # change key names for CPU runtime
        pretrained_weights = get_cpu_model(pretrained_weights)
        # load the pretrained weights into model
        model.load_state_dict(pretrained_weights)

    # change model's mode to evaluation
    model.eval()

    video_path = args.video
    # capture the video and get the first frame
    cap = cv2.VideoCapture(video_path)
    ret, frame_1 = cap.read()
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # frame preprocessing
    frame_1 = cv2.resize(frame_1, (256, 256), interpolation=cv2.INTER_AREA)
    frame_1 = frame_preprocess(frame_1, device)

    counter = 0
    # Define the codec and create VideoWriter object
    with torch.no_grad():
       for i in tqdm(range(length)):
            # read the next frame
            ret, frame_2 = cap.read()
            if not ret:
                break
            # preprocessing
            frame_2 = cv2.resize(frame_2, (256, 256), interpolation=cv2.INTER_AREA)
            frame_2 = frame_preprocess(frame_2, device)
            # predict the flow
            flow_low, flow_up = model(frame_1, frame_2, iters=args.iters, test_mode=True)
            # transpose the flow output and convert it into numpy array
            ret = vizualize_flow(frame_1, flow_up, save, counter, vid_number)
            if not ret:
                break
            frame_1 = frame_2
            counter += 1

In [ ]:
def create_video_opencv(image_folder, video_name, fps, dataset_video=None):
  if dataset_video:
      os.remove(dataset_video)

  images = [img for img in os.listdir(image_folder) if img.endswith(".tif")]
  images.sort()
  frame = cv2.imread(os.path.join(image_folder, images[0]))
  height, width, layers = frame.shape

  video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

  for image in images:
      video.write(cv2.imread(os.path.join(image_folder, image)))

  cv2.destroyAllWindows()
  video.release()

def create_optical_flow(root_dir, video_name, fps, video_number, dataset_video=None):
  create_video_opencv(root_dir, video_name, fps, dataset_video)

  args = argparse.Namespace(
      iters=12, 
      mixed_precision=False, 
      model='/content/drive/MyDrive/optical_flow_RAFT/models/raft-sintel.pth', 
      save=True, 
      small=False, 
      video=video_name,
      video_number=video_number)
  inference(args)

In [ ]:
for i in range(1, 35):
  video_path = (f"/content/drive/MyDrive/AnomalyResearch/Train/Train{str(i).zfill(3)}/")
  create_optical_flow(video_path, "test.avi", 15, i)